## COMP 4437 Week 6
### Scroll down for the questions. 🠋

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io.arff import loadarff
import json
import random

# Load data
raw_data_train = loadarff('emotions-train.arff')
raw_data_test = loadarff('emotions-test.arff')


df_train = pd.DataFrame(raw_data_train[0]).astype(float)
df_test = pd.DataFrame(raw_data_test[0]).astype(float)


labels = ['amazed-suprised', 'happy-pleased', 'relaxing-calm', 'quiet-still', 'sad-lonely', 'angry-aggresive']

X_train = df_train.drop(labels, axis=1)
Y_train = df_train[labels]

X_train_np = X_train.to_numpy()
Y_train_np = Y_train.to_numpy()


X_test = df_test.drop(labels, axis=1)
Y_test = df_test[labels]

X_test_np = X_test.to_numpy()
Y_test_np = Y_test.to_numpy()


def batch_generator(X, Y, batch_size, shuffle=True):
    num_samples = X.shape[0]
    if shuffle:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        X = X[indices]
        Y = Y[indices]
    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        yield (X[start_idx:end_idx], Y[start_idx:end_idx])
        
        

def hyperparameter_save(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values):
    hyperparameters = {
            'epochs': epochs,
            'batch_size': batch_size,
            'hidden_size': hidden_size,
            'learning_rate': learning_rate,
            'reg_lambda': reg_lambda,
            'losses': loss_values
        }
    with open('hyperparameters.json', 'a') as file:
        json.dump(hyperparameters, file)
        file.write('\n')  # Add a newline to separate entries

    read_hyperparameters_list = []
    with open('hyperparameters.json', 'r') as file:
        for line in file:
            hyperparameters_dict = json.loads(line.strip())
            read_hyperparameters_list.append(hyperparameters_dict)

    for hyperparameters in read_hyperparameters_list:
        print(hyperparameters)
           
    return read_hyperparameters_list        


## COMP 4437 Week 5 Lab Questions
---
#### Q1) Music Emotions dataset is given. Each entry of the dataset corresponds to a song. Each song can have multiple labels from:
- amazed-suprised, 
- happy-pleased, 
- relaxing-calm, 
- quiet-still, 
- sad-lonely, 
- angry-aggresive

<b> Implement <b style="color:red">Multi Layer Perceptron</b> with  <b style="color:red">2 Layers</b> for multilabel classification in a python class. Implement training loops and validation loops in the appropriate cells.

<div class="alert alert-block alert-info">
<b>Tip:</b> Check the last week implementation of Two Layer Perceptron for binary classification
</div>
    
<div class="alert alert-block alert-danger">
<b>!!</b> Make sure to use <b>X_train_np, Y_train_np, X_test_np, Y_test_np</b> for training and testing. Check above cell. 
</div> 
    
---    
#### Q2) Implement a searching method to figure out the best hyperparameters for this problem. Make sure to save each test run with the code given. Expected accuracy is atleast  <b style="color:red">35%</b>
    
---    
<br>    
    

In [2]:
class TwoLayerMLP:

    def __init__(self, input_size, hidden_size, output_size, reg_lambda):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.reg_lambda = reg_lambda
        self.init_weights()
        self.zero_grad()

    def init_weights(self):
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * 0.01
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * 0.01
        self.b2 = np.zeros((1, self.output_size))

    def one_hot_encode(self, Y, num_classes):
        one_hot_Y = np.zeros((Y.size, num_classes))
        one_hot_Y[np.arange(Y.size), Y] = 1
        return one_hot_Y   

    def zero_grad(self):
        self.dW1 = np.zeros_like(self.W1)
        self.db1 = np.zeros_like(self.b1)
        self.dW2 = np.zeros_like(self.W2)
        self.db2 = np.zeros_like(self.b2)

    def update_weights(self, learning_rate):
        self.W1 -= learning_rate * self.dW1
        self.b1 -= learning_rate * self.db1
        self.W2 -= learning_rate * self.dW2
        self.b2 -= learning_rate * self.db2

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def sigmoid_derivative(self, Z):
        s = self.sigmoid(Z)
        return s * (1 - s)

    def forward(self, X, Y=None):
        self.X = X
        if Y is not None:
            self.Y = Y
        
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.H1 = self.Z1
        self.Z2 = np.dot(self.H1, self.W2) + self.b2
        self.H2 = self.sigmoid(self.Z2)
        
        return self.H2

    
    
    def compute_loss(self, Y_pred=None, Y=None):
        if Y_pred is None:
            Y_pred = self.H2  
        if Y is None:
            Y = self.Y        
        
        batch_size = Y.shape[0]
        data_loss = np.sum((Y_pred - Y) ** 2) / batch_size
        reg_loss = (self.reg_lambda / 2) * (np.sum(np.square(self.W2)) + np.sum(np.square(self.W1))) / batch_size
        loss = data_loss + reg_loss      
        
        return loss

    def backward(self):
        Y_pred = self.H2 
        Y = self.Y
        X = self.X
        batch_size = Y.shape[0]

        dZ2 = (Y_pred - Y) * self.sigmoid_derivative(self.Z2)
        self.dW2 = (np.dot(self.H1.T, dZ2) + self.reg_lambda * self.W2) / batch_size
        self.db2 = np.sum(dZ2, axis=0, keepdims=True) / batch_size

        dZ1 = np.dot(dZ2, self.W2.T) * self.sigmoid_derivative(self.Z1)
        self.dW1 = (np.dot(X.T, dZ1) + self.reg_lambda * self.W1) / batch_size
        self.db1 = np.sum(dZ1, axis=0, keepdims=True) / batch_size



In [3]:
class SGD:
    #Leave it as is
    def __init__(self, model, learning_rate, batch_size):
        self.model = model
        self.learning_rate = learning_rate
        self.batch_size = batch_size
     
    def step(self):
        self.model.update_weights(self.learning_rate)
        
    def zero_grad(self):
        self.model.zero_grad()
        
    

In [18]:

def train_loop(model, optimizer, X_train, Y_train, epochs, batch_size):
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, Y_batch in batch_generator(X_train, Y_train, batch_size):
            model.forward(X_batch, Y_batch)  
            loss = model.compute_loss(Y_batch)  
            model.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss
        losses.append(total_loss)
    return losses    


def validate_loop(model, X_val, Y_val):    
    Y_pred = model.forward(X_val)
    np.where(Y_pred > 0.5, 1, 0)
    
    
    predictions = np.argmax(Y_pred, axis=1)
    numerator = np.sum(np.logical_and(Y_val,Y_pred))
    demoninator = np.sum(np.logical_or(Y_val,Y_pred))
    accuracy = np.mean(numerator/demoninator)
    
    print(f"Validation Accuracy: {accuracy:.4f}")
    return predictions, Y_val  # Return predictions and true labels
    
    """
    # Use this formula for accuracy calculation
    numerator = np.sum(np.logical_and(Y_val,y_pred))
    demoninator = np.sum(np.logical_or(Y_val,y_pred))
    accuracy = np.mean(numerator/demoninator)
    
    """
    
    return predictions, Y_val


In [24]:
epochs = 100
batch_size = 100
hidden_size = 200
learning_rate = 0.1
reg_lambda = 3
input_size=72
output_size= 6
loss_values = []


#model = TwoLayerMLP(input_size, hidden_size, output_size, reg_lambda) 
#optimizer = SGD(model, learning_rate)
#loss_values=train_loop(model, optimizer, X_train_np, Y_train_np, epochs, batch_size)
#predictions, Y_val = validate_loop(model, X_test_np, Y_test_np)

np.random.seed(42)  # For reproducible results. Do not change


"""
######
Solve Question 2 Here


### Do this for every hyperparameter test
read_hyperparameters_list=hyperparameter_save(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values):


######
"""
read_hyperparameters_list = hyperparameter_save(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values)

def hyper_func(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values):
    model = TwoLayerMLP(input_size, hidden_size, output_size, reg_lambda)
    optimizer = SGD(model, learning_rate, batch_size)
    loss_values = train_loop(model, optimizer, X_train_np, Y_train_np, epochs, batch_size)
    predictions, Y_val = validate_loop(model, X_test_np, Y_test_np)
    
    read_hyperparameters_list.append(hyperparameter_save(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values))


#for hyperparameter in hyperparameters_list:
for epochs in [1,100,50,20]:
    hyper_func(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values)
for batch_size in [1,50,10,20]:    
    hyper_func(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values)
for hidden_size in [10,20,50,100]:   
    hyper_func(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values)
for learning_rate in [0.1, 0.01, 0.2, 0.3]:     
    hyper_func(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values)
for reg_lambda in [1, 3, 0.1, 0.01]:    
    hyper_func(epochs,batch_size,hidden_size,learning_rate,reg_lambda,loss_values)





{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learn

Validation Accuracy: 0.3292
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None,

Validation Accuracy: 0.3292
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None,

Validation Accuracy: 0.3292
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None,

Validation Accuracy: 0.3292
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None,

...

Validation Accuracy: 0.3292
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None,

Validation Accuracy: 0.3292
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None,

{'epochs': 100, 'batch_size': 100, 'hidden_size': 200, 'learning_rate': 0.1, 'reg_lambda': 3, 'losses': 0.004443021592357043}
{'epochs': 150, 'batch_size': 100, 'hidden_size': 200, 'learning_rate': 0.1, 'reg_lambda': 3, 'losses': 0.003651416894581085}
{'epochs': 200, 'batch_size': 100, 'hidden_size': 200, 'learning_rate': 0.1, 'reg_lambda': 3, 'losses': 0.0035053440409378313}
{'epochs': 200, 'batch_size': 1, 'hidden_size': 200, 'learning_rate': 0.1, 'reg_lambda': 3, 'losses': 0.326884606710726}
{'epochs': 200, 'batch_size': 5, 'hidden_size': 200, 'learning_rate': 0.1, 'reg_lambda': 3, 'losses': 0.004969299339959189}
{'epochs': 200, 'batch_size': 10, 'hidden_size': 200, 'learning_rate': 0.1, 'reg_lambda': 3, 'losses': 3.4880363569566466e-08}
{'epochs': 1, 'batch_size': 100, 'hidden_size': 200, 'learning_rate': 0.1, 'reg_lambda': 3, 'losses': 0.028130594881462564}
{'epochs': 100, 'batch_size': 100, 'hidden_size': 200, 'learning_rate': 0.1, 'reg_lambda': 3, 'losses': 0.004443021592357043}

Validation Accuracy: 0.3292
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None,

{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learning_rate': None, 'reg_lambda': None, 'losses': None}
{'epochs': None, 'batch_size': None, 'hidden_size': None, 'learn

In [25]:
"""Run For Plotting each hyperparameter test on a different subplot"""
fig, axes = plt.subplots(len(read_hyperparameters_list), 1, figsize=(8, 6 * len(read_hyperparameters_list)))

for idx, hyperparameters in enumerate(read_hyperparameters_list):
    losses = hyperparameters['losses']
    epochs = hyperparameters['epochs']
    batch_size = hyperparameters['batch_size']
    hidden_size = hyperparameters['hidden_size']
    learning_rate = hyperparameters['learning_rate']
    reg_lambda = hyperparameters['reg_lambda']

    ax = axes[idx] if len(read_hyperparameters_list) > 1 else axes
    ax.plot(losses)
    ax.set_title(f'Epochs: {epochs}, Batch Size: {batch_size}, Hidden Size: {hidden_size}, Learning Rate: {learning_rate}, Reg Lambda: {reg_lambda}')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')

plt.tight_layout()
plt.show()

ValueError: x, y, and format string must not be None

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000001CDB231CC80> (for post_execute):


ValueError: Image size of 576x126144 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 576x126144 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 576x126144 with 292 Axes>

In [ ]:
"""Run For Plotting each hyperparameter test on a single"""
fig, ax = plt.subplots(figsize=(10, 6))

for idx, hyperparameters in enumerate(read_hyperparameters_list):
    losses = hyperparameters['losses']
    epochs = hyperparameters['epochs']
    batch_size = hyperparameters['batch_size']
    hidden_size = hyperparameters['hidden_size']
    learning_rate = hyperparameters['learning_rate']
    reg_lambda = hyperparameters['reg_lambda']

    ax.plot(losses, label=f'Run {idx + 1}')
    ax.text(len(losses) - 1, losses[-1], f'Epochs={epochs}\nBatch Size={batch_size}\nHidden Size={hidden_size}\
    \nLearning Rate={learning_rate}\nReg Lambda={reg_lambda}', ha='center', va='center', fontsize=8)

ax.set_title('Loss Curves for Different Hyperparameter Settings')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.legend()
plt.grid(True)

plt.tight_layout()
plt.show()